<a href="https://colab.research.google.com/github/Biniyamgd/IDP/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#progress

!pip install dlib
!pip install face_recognition
from google.colab import drive
drive.mount('/content/drive')
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("/content/drive/MyDrive/serviceAccountKey.json")
firebase_admin.initialize_app(cred,{
    "databaseURL":"https://facerecondb-default-rtdb.firebaseio.com/",
    "storageBucket":"facerecondb.appspot.com"
})


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=e2f5b784ce092e2eb77ed6cbeebb9387d548fb151c41994c084f04885d93babe
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
Mounted at /content/drive


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

!mkdir unknown

def take_photo(filename='./unknown/photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image, clear_output
from google.colab.patches import cv2_imshow

from threading import local
import cv2
import os
import pickle
import face_recognition
import numpy as np
import firebase_admin
from firebase_admin import storage
from firebase_admin import db
import time
from datetime import datetime

def Retake():
 try:
   filename = take_photo()
  #  print('Saved to {}'.format(filename))

   # Show the image which was just taken.
   # display(Image(filename))  #check
 except Exception as err:
   # Errors will be thrown if the user does not have a webcam or if they do not
   # grant the page permission to access it.
   print(str(err))



 filepath='./drive/MyDrive/students'
 folder=os.listdir(filepath)
 modelist=[]
 studentId=[]


 for Path in folder:
  # Check if the item is a file and a supported image format before adding to modelist
  if os.path.isfile(os.path.join(filepath, Path)) and Path.lower().endswith(('.png', '.jpg', '.jpeg')):
    modelist.append(cv2.imread(os.path.join(filepath,Path)))
    studentId.append(os.path.splitext(Path)[0])
    imgfile=os.path.join(filepath,Path)
    # Get the relative path to avoid including parent directories
    relative_path = os.path.relpath(imgfile, filepath)
    bucket=storage.bucket()
    blob = bucket.blob(relative_path)
    blob.upload_from_filename(imgfile)
    # print(f'File {imgfile} uploaded to {relative_path}.')
#  print(studentId)

 encodeList=[]
 def encodingImg(imgList): # Changed function parameter to imgList to reflect that it's processing a list

    for img in imgList: # Iterate over each image in the list
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img) # Use img instead of i
        encodeList.append(encode)
    return encodeList

#  print('encoding Started...')
 encodes=encodingImg(modelist) # Pass the list of images
 unknownId=[encodes[0],studentId]
#  print('encoding Complete...')
 # print(unknownId)

 file =open('./sample_data/encoding.p','wb')
 pickle.dump(unknownId,file)
 file.close()
#  print('File Saved')

 file = open('./sample_data/encoding.p','rb')
 encodeListknownId=pickle.load(file)
 file.close()
 un , std = encodeListknownId
#  print(std)



 # print(modelist)
 filepath2=cv2.imread(os.path.join('/content/unknown','photo.jpg'))

 def encodingImg2(img): # Changed function parameter to imgList to reflect that it's processing a list
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img) # Use img instead of i
        return encode

 encodehello = encodingImg2(filepath2)
 i=0
 Matches=[]
 FaceDistance=[]

 try:
  while i<len(encodes):
    for items in encodes[i]:
      match1=face_recognition.compare_faces(items,encodehello)
      facedis=face_recognition.face_distance(items,encodehello)
      Matches.append(match1[0])
      FaceDistance.append(facedis[0])
    i=i+1
 except Exception as err:
     print("Retake The Photo Please")
     Retake()
     return
#  print(Matches)
#  print(FaceDistance)
 if True in Matches:
   index=np.argmin(FaceDistance)
   ID=studentId[index]
   IDref=db.reference(f'students/{ID}')
  #  print(IDref.get())
   studentInfo=IDref.get()
   datetimeObject = datetime.strptime(studentInfo['last_attendance_time'],"%Y-%m-%d %H:%M:%S")
   secondsElapsed = (datetime.now() - datetimeObject).total_seconds()
  #  print(secondsElapsed)
   if secondsElapsed > 30:
     studentInfo['Total_attendance'] += 1
     IDref.child('Total_attendance').set(studentInfo['Total_attendance'])
     IDref.child('last_attendance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
     delay=2
     checked=cv2.imread('/content/drive/MyDrive/display/checked.jpg')
     cv2_imshow(checked)
     time.sleep(delay)
     clear_output()


     Downblob=bucket.get_blob(f'{ID}.jpg')
     array=np.frombuffer(Downblob.download_as_string(),np.uint8)
     image1=cv2.imdecode(array,cv2.IMREAD_COLOR)
     # Load the images
     img2 = cv2.imread('/content/drive/MyDrive/display/info.jpg')
     text = f"{studentInfo['name']}"

     # Resize the images to the same size (if needed)
     img1 = cv2.resize(image1, (300, 400))
     img2 = cv2.resize(img2, (300, 400))
     # Define the position, font, scale, color, and thickness of the text
     position = (75, 105)  # (x, y) coordinates for the bottom-left corner of the text
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)  # BGR (Blue, Green, Red)
     thickness = 2
     # Add text to the image
     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"ETS{ID}/13"
     position = (45, 170)  # (x, y) coordinates for the bottom-left corner of the text
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)  # BGR (Blue, Green, Red)
     thickness = 2
     # Add text to the image
     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"{studentInfo['department']}"
     position = (137, 235)  # (x, y) coordinates for the bottom-left corner of the text
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)  # BGR (Blue, Green, Red)
     thickness = 2
     # Add text to the image
     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"{studentInfo['course']}"
     position = (88, 302)  # (x, y) coordinates for the bottom-left corner of the text
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)  # BGR (Blue, Green, Red)
     thickness = 2
     # Add text to the image
     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"{studentInfo['Total_attendance']}"
     position = (185, 360)  # (x, y) coordinates for the bottom-left corner of the text
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)  # BGR (Blue, Green, Red)
     thickness = 2
     # Add text to the image
     cv2.putText(img2, text, position, font, font_scale, color, thickness)



     # Concatenate the images horizontally
     img_combined = cv2.hconcat([img1, img2])
     # Display the combined image
     cv2_imshow(img_combined)
     # Wait for the user to close the window
     # cv2.waitKey(0)
     # cv2.destroyAllWindows()
     delay=10
     time.sleep(delay)
     clear_output()
     Retake()
   else:
      al_img=cv2.imread('/content/drive/MyDrive/display/already.jpg')
      cv2_imshow(al_img)
      delay=5
      time.sleep(delay)
      clear_output()
      Retake()
 else:
    un_img=cv2.imread('/content/drive/MyDrive/display/unregistered.jpg')
    cv2_imshow(un_img)
    delay=5
    time.sleep(delay)
    clear_output()
    Retake()
Retake()

<IPython.core.display.Javascript object>

In [ ]:

# import firebase_admin
# from firebase_admin import credentials
# from firebase_admin import db



# # cred = credentials.Certificate("/content/drive/MyDrive/serviceAccountKey.json")
# # firebase_admin.initialize_app(cred,{
# #     "databaseURL":"https://facerecondb-default-rtdb.firebaseio.com/"
# # })

# ref=db.reference('students')


# data= {
# "1316":{
#     "name":"Yihun Melkam",
#     "major":"Computer",
#     "starting_year":2021,
#     "Total_attendace":9,
#     "Year":4,
#     "last_attendance_time":"2024-02-12 00:00:00"
# },
# "0273":{
#     "name":"Biniyam Gedefaw",
#     "major":"Computer",
#     "starting_year":2021,
#     "Total_attendace":8,
#     "Year":4,
#     "last_attendance_time":"2024-02-12 00:00:00"
# }
# }


# for key,value in data.items():
#     ref.child(key).set(value)


